In [12]:
# Cell 1: Imports and Configuration
import pandas as pd
import numpy as np
import os

# Define file paths for clarity
# This assumes your notebook is in a 'notebooks' folder, parallel to a 'data' folder
DATA_DIR = '../data'
OUTPUT_DIR = '../output/processed_data'

# Create the output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Cell 2: Markdown
"""
# Step 0: Data Preparation and Merging

This notebook serves as the foundational step for the entire project. Its purpose is to:
1. Load the five raw datasets provided by United Airlines.
2. Perform necessary cleaning and aggregations to bring all data to a consistent, flight-level view.
3. Handle specific data inconsistencies found during initial exploration (e.g., column names).
4. Merge all sources into a single, master DataFrame.
5. Save the final, clean dataset to a new CSV file.

All subsequent analytical notebooks will begin by loading the clean data file produced by this notebook. This approach ensures consistency and avoids code repetition.
"""

In [13]:
# Cell 3: Load Raw Data
print("--- Loading all raw data sources ---")
try:
    flight_df = pd.read_csv(os.path.join(DATA_DIR, 'Flight Level Data.csv'))
    # Your PNR file has a '+' in the name, so we use that exact name
    pnr_flight_df = pd.read_csv(os.path.join(DATA_DIR, 'PNR+Flight+Level+Data.csv'))
    pnr_remarks_df = pd.read_csv(os.path.join(DATA_DIR, 'PNR Remark Level Data.csv'))
    # Your Bag file has a '+' in the name
    bag_df = pd.read_csv(os.path.join(DATA_DIR, 'Bag+Level+Data.csv'))
    airports_df = pd.read_csv(os.path.join(DATA_DIR, 'Airports Data.csv'))
    print("All CSV files loaded successfully.")
except FileNotFoundError as e:
    print(f"ERROR: Could not find data file - {e}.")
    flight_df = None

--- Loading all raw data sources ---
All CSV files loaded successfully.


# Cell 4: Markdown
"""
### Data Aggregation
To create a flight-level view, we need to aggregate the PNR, Remarks, and Bag data.

- **PNR Data:** We will sum passenger counts and related flags for each unique flight.
- **Remarks Data:** We will count the number of special service requests (SSRs) for each flight.
- **Bag Data:** We will count the number of 'Checked' and 'Transfer' bags for each flight.
"""

In [14]:
# Cell 4: Data Aggregation and Cleaning
if flight_df is not None:
    flight_key = [
        'scheduled_departure_date_local', 'company_id', 'flight_number',
        'scheduled_departure_station_code', 'scheduled_arrival_station_code'
    ]

    print("Aggregating PNR data...")
    # Your file uses 'basic_economy_ind', we will rename it to be consistent
    if 'basic_economy_ind' in pnr_flight_df.columns:
        pnr_flight_df.rename(columns={'basic_economy_ind': 'basic_economy_pax'}, inplace=True)
    pnr_agg = pnr_flight_df.groupby(flight_key).agg(
        total_pax=('total_pax', 'sum'),
        lap_child_count=('lap_child_count', 'sum'),
        is_child_count=('is_child', lambda x: (x == 'Y').sum()),
        is_stroller_user_count=('is_stroller_user', lambda x: (x == 'Y').sum()),
        basic_economy_pax=('basic_economy_pax', 'sum')
    ).reset_index()

    print("Aggregating Remarks (SSR) data...")
    pnr_key_map = pnr_flight_df[['record_locator'] + flight_key].drop_duplicates()
    remarks_with_flight_key = pd.merge(
        pnr_remarks_df.drop(columns=['flight_number'], errors='ignore'),
        pnr_key_map, on='record_locator', how='left'
    )
    ssr_agg = remarks_with_flight_key.groupby(flight_key).size().reset_index(name='ssr_count')

    print("Aggregating Bag data...")
    bag_agg = bag_df.pivot_table(
        index=flight_key, columns='bag_type', values='bag_tag_unique_number',
        aggfunc='count', fill_value=0
    ).reset_index()
    bag_agg.columns.name = None
    # Your file uses 'Origin', we will rename it to the standard 'Checked'
    if 'Origin' in bag_agg.columns:
        bag_agg.rename(columns={'Origin': 'Checked'}, inplace=True)
    if 'Transfer' not in bag_agg.columns:
        bag_agg['Transfer'] = 0

Aggregating PNR data...
Aggregating Remarks (SSR) data...
Aggregating Bag data...


# Cell 4: Markdown
"""
### Data Aggregation
To create a flight-level view, we need to aggregate the PNR, Remarks, and Bag data.

- **PNR Data:** We will sum passenger counts and related flags for each unique flight.
- **Remarks Data:** We will count the number of special service requests (SSRs) for each flight.
- **Bag Data:** We will count the number of 'Checked' and 'Transfer' bags for each flight.
"""

In [15]:
# Cell 5: Merge and Create Master DataFrame
print("\n--- Merging all data sources into a single master DataFrame ---")
master_df = pd.merge(flight_df, pnr_agg, on=flight_key, how='left')
master_df = pd.merge(master_df, ssr_agg, on=flight_key, how='left')
master_df = pd.merge(master_df, bag_agg, on=flight_key, how='left')

# Fill NaNs in count columns with 0
count_cols = [
    'total_pax', 'lap_child_count', 'is_child_count', 'basic_economy_pax',
    'is_stroller_user_count', 'ssr_count', 'Checked', 'Transfer'
]
for col in count_cols:
    if col in master_df.columns:
        master_df[col] = master_df[col].fillna(0)


--- Merging all data sources into a single master DataFrame ---


In [16]:
# Cell 6: Foundational Feature Engineering
print("\n--- Creating all foundational features needed for subsequent analyses ---")

# Date and Delay Features
master_df['actual_departure_datetime_local'] = pd.to_datetime(master_df['actual_departure_datetime_local'], errors='coerce')
master_df['scheduled_departure_datetime_local'] = pd.to_datetime(master_df['scheduled_departure_datetime_local'], errors='coerce')
master_df['departure_delay'] = (master_df['actual_departure_datetime_local'] - master_df['scheduled_departure_datetime_local']).dt.total_seconds() / 60

# Ground Time Feature
master_df['ground_time_deficit'] = master_df['scheduled_ground_time_minutes'] - master_df['minimum_turn_minutes']

# Passenger-based Features
master_df['load_factor'] = master_df['total_pax'] / master_df['total_seats'].replace(0, pd.NA)
master_df['ssr_per_pax'] = master_df['ssr_count'] / master_df['total_pax'].replace(0, pd.NA)
master_df['child_ratio'] = (master_df['lap_child_count'] + master_df['is_child_count']) / master_df['total_pax'].replace(0, pd.NA)
master_df['basic_economy_ratio'] = master_df['basic_economy_pax'] / master_df['total_pax'].replace(0, pd.NA)

# Baggage-based Features
master_df['total_bags'] = master_df['Checked'] + master_df['Transfer']
master_df['bags_per_pax'] = master_df['total_bags'] / master_df['total_pax'].replace(0, pd.NA)

# Flight Profile Feature
airports_df.rename(columns={'airport_iata_code': 'scheduled_arrival_station_code'}, inplace=True)
master_df = pd.merge(master_df, airports_df[['scheduled_arrival_station_code', 'iso_country_code']], on='scheduled_arrival_station_code', how='left')
master_df['is_international'] = (master_df['iso_country_code'] != 'US').astype(int)

# Final cleanup
master_df.fillna(0, inplace=True)
print("All foundational features have been created successfully.")


--- Creating all foundational features needed for subsequent analyses ---
All foundational features have been created successfully.


In [17]:
# Cell 7: Save the Final Analysis-Ready Data
output_path = os.path.join(OUTPUT_DIR, 'analysis_ready_data.csv')
master_df.to_csv(output_path, index=False)

print(f"\n✅ Analysis-ready data saved to: {output_path}")
print("All other notebooks should now load this single file.")


✅ Analysis-ready data saved to: ../output/processed_data\analysis_ready_data.csv
All other notebooks should now load this single file.
